In [2]:
# install PyDrive
!pip install -U -q PyDrive

# imports
from __future__ import print_function
import os
import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import lite
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Flatten, Add, Concatenate, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras import backend as K
from scipy.signal import stft
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 35.3MB/s 


In [0]:
# saved files names
img_model_name = "keras_cnn_img.h5"
emg_model_name = "keras_cnn_emg.h5"
fus_model_name = "keras_cnn_fus.h5"

# hyper-parameters definition
split_p = 0.66
batch_size = 128
num_classes = 5
img_epochs = 30
emg_epochs = 30
fus_epochs = 30
img_rows, img_cols = 60, 60
emg_rows, emg_cols = 8, 3
seed = 7
np.random.seed(seed)

In [0]:
# authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link_img = "https://drive.google.com/open?id=1MLV46ljNKlA83O61l1w7MPKUm54XMxkr"
link_emg = "https://drive.google.com/open?id=19DObL7KWciZ4CpGzX_yAWqmBu0DiwJgG"
link_lbl = "https://drive.google.com/open?id=1n-qxQ7d64MkfnSPqE1z0qCCy6zYvZaBb"


In [0]:
# load data files
fluff, id = link_img.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({"id":id}) 
downloaded.GetContentFile("cc19_hand_gestures_10p_img.npy")  
data_img = np.load("cc19_hand_gestures_10p_img.npy")
print("data_img.shape = ", data_img.shape)

fluff, id = link_emg.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({"id":id}) 
downloaded.GetContentFile("cc19_hand_gestures_10p_emg.npy")  
data_emg = np.load("cc19_hand_gestures_10p_emg.npy")
print("data_emg.shape = ", data_emg.shape)

fluff, id = link_lbl.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({"id":id}) 
downloaded.GetContentFile("cc19_hand_gestures_10p_lbl.npy")  
labels = np.load("cc19_hand_gestures_10p_lbl.npy")
print("labels.shape = ", labels.shape)

1MLV46ljNKlA83O61l1w7MPKUm54XMxkr
data_img.shape =  (5600, 60, 60)
19DObL7KWciZ4CpGzX_yAWqmBu0DiwJgG
data_emg.shape =  (5600, 40, 8)
1n-qxQ7d64MkfnSPqE1z0qCCy6zYvZaBb
labels.shape =  (5600, 1)


In [0]:
# function to extract emg features
def analyze(x, fs=200, frame_len=0.2, frame_step=0.1, feat='MAV', order=1, threshold=0.1, preprocess=True):
    l, n_ch = x.shape
    frame_len_samples = int(frame_len * fs)
    frame_step_samples = int(frame_step * fs)
    
    n_win = (l - frame_len_samples) // frame_step_samples + 1
    ret = np.zeros((n_win, n_ch))
    
    w = np.hamming(frame_len_samples)
    
    if preprocess:
        filteredEMGSignal = butter_lowpass_filter(x.T, 99, fs, 2)
        x = butter_highpass_filter(filteredEMGSignal, 20, fs, 2).T

        
    for i in range(n_win):
        chunk = x[i * frame_step_samples: i * frame_step_samples + frame_len_samples]
        
        # TIME DOMAIN
        if feat == 'MAV':
            ret[i] = np.mean(np.abs(chunk), 0)
        if feat == 'iEMG':
            ret[i] = np.sum(np.abs(chunk), 0)
        if feat == 'SSI':
            ret[i] = np.sum(chunk ** 2, 0)
        if feat == 'VAR':
            ret[i] = np.sum(chunk ** 2, 0) / (frame_len_samples - 1)
        if feat == 'SD':
            ret[i] = np.std(chunk, 0)
        if feat == 'TM':
            ret[i] = np.abs(np.sum(chunk ** order, 0) / frame_len_samples)
        if feat == 'RMS':
            ret[i] = np.sqrt(np.sum(chunk ** 2, 0) / frame_len_samples)
        if feat == 'LOG':
            ret[i] = np.exp(np.sum(np.abs(chunk), 0) / frame_len_samples) 
        if feat == 'WL':
            ret[i] = np.sum(np.abs(np.diff(chunk.T).T), 0)
        if feat == 'AAC':
            ret[i] = np.sum(np.abs(np.diff(chunk.T).T), 0) / frame_len_samples
        if feat == 'DASDV':
            ret[i] = np.sum(np.diff(chunk.T).T ** 2, 0) / (frame_len_samples - 1)
        if feat == 'AFB':
            for j in range(n_ch):
                _s = np.convolve(w / w.sum(), chunk[:, j] ** 2, mode='valid')
                pp = peakutils.indexes(_s)
                if len(pp) > 0:
                    peak = pp[0]
                else:
                    peak = 0
                ret[i, j] = _s[peak]
        if feat == 'ZC':
            positive = [chunk[0, i] > threshold for i in range(n_ch)]
            ZC = np.zeros((n_ch,))
            for j in range(n_ch):
                for ss in chunk[1:, j]:
                    if positive[j]:
                        if(ss < 0 -threshold):
                            positive[j] = False
                            ZC[j] += 1
                    else:
                        if(ss > 0 + threshold):
                            positive[j] = True
            ret[i] = ZC
        if feat == 'MYOP':
            ret[i] = np.sum(np.float32(chunk >= threshold), 0) / frame_len_samples
        if feat == 'WAMP':
            ret[i] = np.sum(np.float32(np.diff(chunk.T).T > threshold))
        if feat == 'SSC':
            N = len(chunck)
            SSC = 0
            for i in range(1,N-1):
                a, b, c = [chunk[i-1], chunk[i], chunk[i+1]]
                if(a + b + c >= threshold * 3 ): #computed only if the 3 values are above the threshold
                    if(a < b > c or a > b < c ): #if there's change in the slope
                        SSC += 1
            ret[i] = SSC
            
        # FREQ DOMAIN
        
    return ret

In [0]:
# emg feature extraction
all_emg = []
frame_len = 0.2
frame_step = 0.1
features = ['MAV', 'RMS', 'SD']
print("data_emg.shape = ", data_emg.shape)
for emg in data_emg:
    sub = [analyze(emg, frame_len=frame_len, frame_step=frame_step, feat=f, preprocess=False) for f in features]
    sub = np.hstack(sub)
    all_emg.append(sub)
data_emg = np.vstack(all_emg)

print("data_emg.shape", data_emg.shape)

data_emg.shape =  (5600, 40, 8)
data_emg.shape (5600, 24)


In [0]:
# function to split data
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]

# shuffle data
data_img, data_emg, labels = unison_shuffled_copies(data_img, data_emg, labels)

# split data
split = int(split_p*len(labels))
print("split = ", split)

data_img_train = data_img[:split,:,:]
data_emg_train = data_emg[:split,:]
data_img_test = data_img[split:,:,:]
data_emg_test = data_emg[split:,:]
labels_train = labels[:split,:]
labels_test = labels[split:,:]

print("data_img_train.shape = ", data_img_train.shape)
print("data_emg_train.shape = ", data_emg_train.shape)

split =  3696
data_img_train.shape =  (3696, 60, 60)
data_emg_train.shape =  (3696, 24)


In [0]:
# img preprocessing
# normalize
data_max = np.max(data_img_train)
data_min = np.min(data_img_train)
for i in range(len(data_img_train)):
    data_img_train[i] = (data_img_train[i] - data_min) / (data_max - data_min)
for i in range(len(data_img_test)):
    data_img_test[i] = (data_img_test[i] - data_min) / (data_max - data_min)
print("img_min = ", data_min)
print("img_max = ", data_max)

# standardize
data_mean = np.mean(data_img_train)
data_std = np.std(data_img_train) + 1e-15
data_img -= data_mean
data_img /= data_std
print("img_mean = ", data_mean)
print("img_std = ", data_std)

# emg preprocessing
# normalize
data_max = np.max(data_emg_train)
data_min = np.min(data_emg_train)
for i in range(len(data_emg_train)):
    data_emg_train[i] = (data_emg_train[i] - data_min) / (data_max - data_min)
for i in range(len(data_emg_test)):
    data_emg_test[i] = (data_emg_test[i] - data_min) / (data_max - data_min)
print("emg_min = ", data_min)
print("emg_max = ", data_max)

# standardize
data_mean = np.mean(data_emg_train)
data_std = np.std(data_emg_train) + 1e-15
data_emg -= data_mean
data_emg /= data_std
print("emg_mean = ", data_mean)
print("emg_std = ", data_std)

img_min =  -10.0
img_max =  4.342945178152237e-11
img_mean =  0.11297054755424565
img_std =  0.29921494048849095
emg_min =  0.0
emg_max =  68.3781763430409
emg_mean =  0.1490529122635982
emg_std =  0.11991758770519481


In [0]:
# data format convertion
data_img_train = data_img_train.reshape(data_img_train.shape[0], img_rows, img_cols, 1)
data_img_test = data_img_test.reshape(data_img_test.shape[0], img_rows, img_cols, 1)
data_emg_train = data_emg_train.reshape(data_emg_train.shape[0], emg_cols*emg_rows, 1)
data_emg_test = data_emg_test.reshape(data_emg_test.shape[0], emg_cols*emg_rows, 1)
labels_train = keras.utils.to_categorical(labels_train, num_classes)
labels_test = keras.utils.to_categorical(labels_test, num_classes)
img_input_shape = (img_rows, img_cols, 1)
emg_input_shape = (emg_cols*emg_rows, 1)

In [0]:
# cnn for img

# create the cnn model
model_img = Sequential()
model_img.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', input_shape=img_input_shape))
model_img.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu'))
model_img.add(MaxPooling2D(pool_size=(2, 2)))
model_img.add(Dropout(0.25))
model_img.add(Flatten())
model_img.add(Dense(50, activation='relu'))
model_img.add(Dropout(0.5))
model_img.add(Dense(num_classes, activation='softmax'))
                    
model_img.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 58, 8)         80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 8)         584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)      

In [0]:
# compile model
model_img.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adadelta(),
            metrics=['accuracy'])

# fit the model
model_img.fit(data_img_train, labels_train,
        batch_size=batch_size,
        epochs=img_epochs,
        verbose=1,
        validation_data=(data_img_test, labels_test))

# evaluate the model
scores = model_img.evaluate(data_img_test, labels_test, verbose=1)
print("%s: %.2f%%" % (model_img.metrics_names[1], scores[1]*100))

Train on 3696 samples, validate on 1904 samples
Epoch 1/30
3696/3696 [==============================] - 1s 335us/step - loss: 0.2524 - acc: 0.8937 - val_loss: 0.2940 - val_acc: 0.8992
Epoch 2/30
3696/3696 [==============================] - 1s 191us/step - loss: 0.2212 - acc: 0.9042 - val_loss: 0.3353 - val_acc: 0.8950
Epoch 3/30
3696/3696 [==============================] - 1s 184us/step - loss: 0.2012 - acc: 0.9175 - val_loss: 0.3488 - val_acc: 0.8992
Epoch 4/30
3696/3696 [==============================] - 1s 182us/step - loss: 0.1993 - acc: 0.9142 - val_loss: 0.3173 - val_acc: 0.9060
Epoch 5/30
3696/3696 [==============================] - 1s 180us/step - loss: 0.1824 - acc: 0.9226 - val_loss: 0.3067 - val_acc: 0.9107
Epoch 6/30
3696/3696 [==============================] - 1s 181us/step - loss: 0.1827 - acc: 0.9221 - val_loss: 0.3490 - val_acc: 0.9007
Epoch 7/30
3696/3696 [==============================] - 1s 180us/step - loss: 0.1923 - acc: 0.9161 - val_loss: 0.3303 - val_acc: 0.9065


In [0]:
# confusion matrix with img
pred_img = model_img.predict(data_img_test)
print(confusion_matrix(labels_test.argmax(axis=1), pred_img.argmax(axis=1)))

[[360   8   6   5  12]
 [ 11 327   2  19   8]
 [ 22   3 351   2  19]
 [  7   4   4 342   7]
 [ 10   3  10   5 357]]


In [0]:
# cnn for emg

# create the cnn model
model_emg = Sequential()
model_emg.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=emg_input_shape))
model_emg.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_emg.add(Dropout(0.5))
model_emg.add(MaxPooling1D(pool_size=2))
model_emg.add(Flatten())
model_emg.add(Dense(100, activation='relu'))
model_emg.add(Dense(num_classes, activation='softmax'))

model_emg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 22, 32)            128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 32)            3104      
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 32)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               32100     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 505       
Total para

In [1]:
# compile model
with session.as_default():
    with session.graph.as_default():
        model_emg.compile(loss=keras.losses.categorical_crossentropy,
                    optimizer=keras.optimizers.Adadelta(),
                    metrics=['accuracy'])

        # fit the model
        model_emg.fit(data_emg_train, labels_train, epochs=emg_epochs, batch_size=batch_size, verbose=0)

        model_emg.fit(data_emg_train, labels_train,
                batch_size=batch_size,
                epochs=emg_epochs,
                verbose=1,
                validation_data=(data_emg_test, labels_test))

        # evaluate the model
        scores = model_emg.evaluate(data_emg_test, labels_test, verbose=0)
        print("%s: %.2f%%" % (model_emg.metrics_names[1], scores[1]*100))

NameError: name 'session' is not defined

In [0]:
# confusion matrix for emg
pred_emg = model_emg.predict(data_emg_test)
print(confusion_matrix(labels_test.argmax(axis=1), pred_emg.argmax(axis=1)))

[[335   3  23  27   3]
 [  2 313  19   8  25]
 [ 37  41 284   7  28]
 [ 11  24   4 314  11]
 [  2  38  13  17 315]]


In [0]:
# mlp for fus

mergedOut = Concatenate()([model_img.output,model_emg.output])
mergedOut = Dense(5, activation='softmax')(mergedOut)
model_fus = Model([model_img.input, model_emg.input], mergedOut)
model_fus.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_1_input (InputLayer)     (None, 60, 60, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 58, 8)    80          conv2d_1_input[0][0]             
__________________________________________________________________________________________________
conv1d_1_input (InputLayer)     (None, 24, 1)        0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 8)    584         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_1 (

In [0]:
# freeze the layers except the last dense
for layer in model_fus.layers[:len(model_fus.layers)-1]:
    layer.trainable = False

# check the status of the layers
for layer in model_fus.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f7ce9419518> False
<keras.layers.convolutional.Conv2D object at 0x7f7cebee6e80> False
<keras.engine.input_layer.InputLayer object at 0x7f7ccfb43c18> False
<keras.layers.convolutional.Conv2D object at 0x7f7ce8e81940> False
<keras.layers.convolutional.Conv1D object at 0x7f7ccfb43320> False
<keras.layers.pooling.MaxPooling2D object at 0x7f7ce8e82c18> False
<keras.layers.convolutional.Conv1D object at 0x7f7ccfb439b0> False
<keras.layers.core.Dropout object at 0x7f7ce94196a0> False
<keras.layers.core.Dropout object at 0x7f7ccfb432b0> False
<keras.layers.core.Flatten object at 0x7f7ce8643e80> False
<keras.layers.pooling.MaxPooling1D object at 0x7f7ccfb52198> False
<keras.layers.core.Dense object at 0x7f7ce8e82cc0> False
<keras.layers.core.Flatten object at 0x7f7ccfb52a90> False
<keras.layers.core.Dropout object at 0x7f7ce8643b70> False
<keras.layers.core.Dense object at 0x7f7ccfb10978> False
<keras.layers.core.Dense object at 0x7f7ce8ee59e8> 

In [0]:
# compile model
model_fus.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adadelta(),
            metrics=['accuracy'])

# fit the model
model_fus.fit([data_img_train, data_emg_train], labels_train,
        batch_size=batch_size,
        epochs=fus_epochs,
        verbose=1,
        validation_data=([data_img_test, data_emg_test], labels_test))

# evaluate the model
scores = model_fus.evaluate([data_img_test, data_emg_test], labels_test, verbose=1)
print("%s: %.2f%%" % (model_fus.metrics_names[1], scores[1]*100))

Train on 3696 samples, validate on 1904 samples
Epoch 1/30
3696/3696 [==============================] - 1s 290us/step - loss: 0.0766 - acc: 0.9805 - val_loss: 0.1569 - val_acc: 0.9433
Epoch 2/30
3696/3696 [==============================] - 0s 128us/step - loss: 0.0664 - acc: 0.9878 - val_loss: 0.1566 - val_acc: 0.9433
Epoch 3/30
3696/3696 [==============================] - 1s 137us/step - loss: 0.0765 - acc: 0.9813 - val_loss: 0.1562 - val_acc: 0.9443
Epoch 4/30
3696/3696 [==============================] - 0s 133us/step - loss: 0.0783 - acc: 0.9781 - val_loss: 0.1559 - val_acc: 0.9443
Epoch 5/30
3696/3696 [==============================] - 1s 136us/step - loss: 0.0713 - acc: 0.9832 - val_loss: 0.1554 - val_acc: 0.9449
Epoch 6/30
3696/3696 [==============================] - 0s 129us/step - loss: 0.0739 - acc: 0.9816 - val_loss: 0.1550 - val_acc: 0.9449
Epoch 7/30
3696/3696 [==============================] - 0s 132us/step - loss: 0.0725 - acc: 0.9808 - val_loss: 0.1546 - val_acc: 0.9449


In [0]:
# confusion matrix for fus
pred_fus = model_fus.predict([data_img_test, data_emg_test])
print(confusion_matrix(labels_test.argmax(axis=1), pred_fus.argmax(axis=1)))

[[374   4   4   3   6]
 [  6 340   3  15   3]
 [ 16   5 366   1   9]
 [  5   3   2 350   4]
 [  6   2   4   4 369]]


In [0]:
# save keras model or weights on google drive
# create on colab directory
model_fus.save(fus_model_name)    
model_file = drive.CreateFile({'title' : fus_model_name})
model_file.SetContentFile(fus_model_name)
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1hNvHUsRRE3tuCxDJSX0ZJd25XkUuaNUj'})

In [6]:
# load model from google drive
link_cnn_fus = "https://drive.google.com/open?id=1hNvHUsRRE3tuCxDJSX0ZJd25XkUuaNUj"

fluff, id = link_cnn_fus.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({"id":id}) 
downloaded.GetContentFile(fus_model_name)  
model_cnn_fus = load_model(fus_model_name)

model_cnn_fus.summary()

1hNvHUsRRE3tuCxDJSX0ZJd25XkUuaNUj
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_1_input (InputLayer)     (None, 60, 60, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 58, 8)    80          conv2d_1_input[0][0]             
__________________________________________________________________________________________________
conv1d_1_input (InputLayer)     (None, 24, 1)        0                                            
______________________________________

In [21]:
# convert h5 model to tflite model
converter = lite.TFLiteConverter.from_keras_model_file(fus_model_name)
model_h5 = converter.convert()
file = open('cnn_fus.tflite','wb')
file.write(model_h5)

# save tflite model on google drive
# create on colab directory
model_file = drive.CreateFile({'title': 'cnn_fus.tflite'})
model_file.SetContentFile('cnn_fus.tflite')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

INFO:tensorflow:Froze 18 variables.
INFO:tensorflow:Converted 18 variables to const ops.


GoogleDriveFile({'id': '1D70qBJzzh3fSaw8oo0pPTOYHJLlH_4sC'})